In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import numpy as np
import re
from langchain.schema import Document
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [12]:

#Importing all documents from the folder - artifacts and cleaning - /n removed, - replaced. 
path_to_pdfs = (r"D:\LLM\projects\Questionme\artifacts")
loader = PyPDFDirectoryLoader(path_to_pdfs)
pdf_documents = loader.load()

def cleanup(document):
    document = document.replace('\n', ' ')
    document = document.replace('-', '')
    document = ' '.join(document.split())
    document = re.sub(r'\n+', ' ', document)
    return document

cleaned_documents = [cleanup(doc.page_content) for doc in pdf_documents]
cleaned_document_objects = [Document(page_content=cleaned_text, metadata=doc.metadata) 
                             for cleaned_text, doc in zip(cleaned_documents, pdf_documents)]
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
final_documents=text_splitter.split_documents(cleaned_document_objects)
texts = [doc.page_content for doc in final_documents]
text = texts


In [13]:
len(text)

109

In [14]:
llm = CTransformers(
    model=r"D:\LLM\models\llama-2-7b-chat.ggmlv3.q8_0.bin",
    model_type="llama",
    config={'max_new_tokens': 300, 'temperature': 0.03,'context_length': 1000}
)

In [16]:

huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True})

c:\Users\Jarviz_92\anaconda3\envs\llm_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:


embeddings = [huggingface_embeddings.embed_query(text) for text in texts]
embeddings_array = np.array(embeddings)
print(embeddings_array)

[[ 0.06778146  0.01100297  0.02574205 ...  0.00751806  0.06441373
  -0.03051739]
 [-0.0693738  -0.02106547 -0.03994767 ... -0.00298001  0.05587736
   0.04344444]
 [-0.03051266  0.08318012 -0.02475407 ... -0.06984834  0.04987878
   0.01845735]
 ...
 [ 0.05832506  0.07038648  0.02622284 ...  0.07168836 -0.01516775
  -0.02078729]
 [ 0.08588448  0.01911633  0.0015234  ...  0.04203937  0.02636152
  -0.02784703]
 [-0.01472853 -0.00520878 -0.00508864 ...  0.07673403 -0.00523054
  -0.04715177]]


In [18]:
vectorstore = FAISS.from_documents(final_documents, huggingface_embeddings)

In [10]:


# Define the prompt template
template = """
You are an expert in UK parking rules, penalties, and fixed penalty details. Answer the question precisely. If you're not 100 percent sure of the answer, respond with 'not sure' and request the user to check updated and correct information or reach a trustworthy solicitor.

Context: {context}

Question: {question}

Answer:
"""

qa_prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={'prompt': qa_prompt}
)

user_input = "Who sets the level of FPN's"

# Retrieve context from the retriever using the correct method
retrieved_docs = retriever.get_relevant_documents(user_input)  # Correct method usage

# Define a function to truncate text to a maximum number of tokens
def truncate_to_token_limit(text, max_tokens):
    tokens = text.split()  # Simple tokenization
    return ' '.join(tokens[:max_tokens]) if len(tokens) > max_tokens else text

# Set a maximum token limit for truncating documents
max_tokens_per_doc = 100  # Example limit for each document

# Truncate each document's content to the specified token limit
truncated_context = [truncate_to_token_limit(doc.page_content, max_tokens_per_doc) for doc in retrieved_docs]

# Calculate the total token count for all inputs
total_tokens = len(user_input.split()) + sum(len(doc.split()) for doc in truncated_context)

# Define the maximum total tokens allowed
max_total_tokens = 512

# Adjust context to ensure total tokens do not exceed the maximum
if total_tokens > max_total_tokens:
    # Truncate further if needed to fit within the total token limit
    truncated_context = truncated_context[:max_total_tokens - len(user_input.split())]

# Ensure the final context is within the allowed token limit
final_context = ' '.join(truncated_context)

# Check again to avoid exceeding the token limit
if len(final_context.split()) + len(user_input.split()) > max_total_tokens:
    print("Final context exceeds maximum token limit. Adjusting to fit.")
    final_context = truncate_to_token_limit(final_context, max_total_tokens - len(user_input.split()))

# Call the chain with the query and the truncated context
try:
    result = chain({'query': user_input, 'context': final_context})  # Correct input keys
    print(f"Answer: {result['result']}")
except Exception as e:
    print(f"Error: {str(e)}")



Answer: Not sure. I'm just an AI and do not have access to the most up-to-date information on UK parking rules, penalties, and fixed penalty details. It is important to consult official sources such as the Rural Payments Agency or the GOV.UK website for the most accurate and reliable information. Additionally, if you have any questions or concerns about enforcement or penalties procedures, you may contact the Meat Technical Schemes team at the Rural Payments Agency directly.
